In [1]:
import pandas as pd
import os
import time
import numpy as np
import torch
import gc

In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
import seaborn as sns
from scipy import stats
from statistics import NormalDist
import matplotlib.pyplot as plt

# Data Engineering

In [4]:
def c_eleuther( returnModel = False, model_type =  'EleutherAI/gpt-neo-125m'):
    ''' Eleuther and Salesforce and Parrot uses the same importation'''
    from transformers import AutoTokenizer, AutoModelForCausalLM
    tokenizer = AutoTokenizer.from_pretrained(model_type)
    logging.info("Tokenizer Loaded")
    if returnModel:
        model = AutoModelForCausalLM.from_pretrained(model_type)
        logging.info("Model Loaded")
    else:
        model = []
    
    logging.info(model_type)
    return tokenizer, model
    
    

def init_model_args( current_case = 'c1', returnModel = False ): 
    
    code_models = {
        'c1':('EleutherAI/gpt-neo-125m', ), # Basic (on Pile) GPT-3/J
        'c2':('EleutherAI/gpt-neo-1.3B', ),
        'c3':('EleutherAI/gpt-neo-2.7B', ),
        'c4':('EleutherAI/gpt-j-6b', ),
        'c5':('Salesforce/codegen-350M-nl', ), #Basic (on Pile) codegen
        'c6':('Salesforce/codegen-2B-nl', ),
        'c7':('Salesforce/codegen-6B-nl', ),
        'c8':('Salesforce/codegen-16B-nl', ),
        'c9':('codeparrot/codeparrot-small-multi', ), #multi-Language
        'c10':('Salesforce/codegen-350M-multi', ),
        'c11':('Salesforce/codegen-2B-multi', ),
        'c12':('Salesforce/codegen-6B-multi', ),
        'c13':('Salesforce/codegen-16B-multi', ),
        'c14':('codeparrot/codeparrot-small', ), #mono-Language
        'c15':('codeparrot/codeparrot', ),
        'c16':('Salesforce/codegen-350M-mono', ),
        'c17':('Salesforce/codegen-2B-mono', ),
        'c18':('Salesforce/codegen-6B-mono', ),
        'c19':('Salesforce/codegen-16B-mono', ),
    }
    
    model_type = code_models[current_case][0]
    tokenizer, model =  c_eleuther( returnModel = returnModel,  model_type = model_type ) 
    
    
    return model_type, tokenizer, model

## Init Parameters

In [5]:
# [WARNING] Hyperparameters changes, please tune them up
params = {
    'codemodel' : 'c2',
    'numpy_files_logits_path': '../datax/np/c',
    'testbeds_path' : '../datax/testbeds/AstEvalVerticalFiltered.json',
    'outputs' : '../datax/ds_raw_logits/out_astevalverticalfiltered_c2.csv'
}

In [6]:
#Uploading Model UnderAnalisys
name, tokenizer, model = init_model_args(
    current_case = params['codemodel'], 
    returnModel = False #[WARNING!] Check the parameters before calling it. 
    )

2023-04-20 21:36:37,588 : INFO : Tokenizer Loaded
2023-04-20 21:36:37,589 : INFO : EleutherAI/gpt-neo-1.3B


In [7]:
#Testing data loads
data_pd = pd.read_json( params['testbeds_path'] )
data_pd.head(2)

,size,ids,m_name,code,ast_errors,n_ast_errors,ast_levels,n_whitespaces_,complexity,nloc,token_counts,n_ast_nodes
0,280,"[4299, 1057, 62, 29412, 62, 41989, 7, 9288, 62...",c1,"def run_python_tests(test_modules, parallelism...",[],0,13,67,7,15,120,154


# Softmax Normalization and Data Engineering

In [9]:
filtered_prompts_ids = data_pd[data_pd['m_name']==params['codemodel']]['ids'].values

In [11]:
tf_input_ids = [torch.tensor(  input_ids, dtype = torch.int) for input_ids in filtered_prompts_ids ]

### Testing Tokenizer

In [13]:
#Testung Sample to validate minmax functions
sample_id = 0
data_pd[data_pd['m_name']==params['codemodel']].iloc[sample_id]

size                                                            280
ids               [4299, 1057, 62, 29412, 62, 41989, 7, 9288, 62...
m_name                                                           c2
code              def run_python_tests(test_modules, parallelism...
ast_errors                                                       []
n_ast_errors                                                      0
ast_levels                                                       13
n_whitespaces_                                                   67
complexity                                                        7
nloc                                                             15
token_counts                                                    120
n_ast_nodes                                                     154
Name: 50971, dtype: object

In [14]:
tf_input_ids[sample_id]

tensor([ 4299,  1057,    62, 29412,    62, 41989,     7,  9288,    62, 18170,
           11, 10730,  1042,    11,   351,    62,  1073,  1857,    28, 25101,
         2599,   198,   220,   220,   220,   900,    62,  7839,    62,   392,
           62,  9967,  7203, 28768,  9485,  4561,   668,  5254,  1600,   366,
         9148, 11290,    62,    47,    56,  4303, 14175,    62,  4944,  2043,
           62,    51,  1546,  4694,  4943,   628,   220,   220,   220,   611,
          351,    62,  1073,  1857,    25,   198,   220,   220,   220,   220,
          220,   220,   220,  1303, 33998,  1838,   262,  9485,  4561,   668,
         5254,   781, 15492,  2233,   284,  4334, 10730,  1042,    13,   198,
          220,   220,   220,   220,   220,   220,   220,  1303,  1649,   356,
         1057,  9485,  4561,   668,  5254,   351,  5197,    11,   340,  3544,
          604,   329,   783,   355,   198,   220,   220,   220,   220,   220,
          220,   220,  1303, 46513,    13,   198,   220,   220, 

In [15]:
# Loading Logits From Files
CODEMODEL =  params['codemodel']
out= np.load( params['numpy_files_logits_path'] +'/' + f'logits_tensor[{sample_id}]_batch[{sample_id}_model[{CODEMODEL}]].npy')
soft = torch.nn.Softmax( dim = 0 ) #Flattening normalization
first_token_distribution = soft( torch.from_numpy(out[0]) ) #Flattening normalization for fist token
assert round(first_token_distribution.sum().item()) == 1.0


FileNotFoundError: [Errno 2] No such file or directory: '../datax/np/c/logits_tensor[0]_batch[0_model[c2]].npy'

In [43]:
first_token_distribution[0]

tensor(0.0013)

In [26]:
print( len(tokenizer.get_vocab())-1, out.shape[1] )
assert len(tokenizer.get_vocab())-1 == out.shape[1]

50001 50001


In [14]:
tokenizer.decode([7881])

' register'

In [15]:
tokenizer.decode([4299,62])

'def_'

In [46]:
tokenizer.decode(4299)

'def'

In [37]:
tokenizer.vocab['<pad>']

50257

In [17]:
tokenizer.vocab.keys()

dict_keys(['Ġveiled', 'Ġresidents', 'ĠAcid', 'iazep', 'ĠSanford', 'perhaps', 'Ġembodiments', 'Ġmaths', 'Ġasymm', '998', 'Allows', 'ĠChau', 'ĠEve', 'ĠCzech', 'high', 'Ġinvol', 'Ġaccomp', 'dec', 'Ġgoodness', 'anut', 'ĠChick', 'ĠOPEN', 'esis', 'ĠPlay', 'Ġobstacle', 'sense', 'ĠPublisher', 'ĠHick', 'ister', 'ĠPalestinians', 'Ġdistinctly', 'pload', 'ĠAS', 'Ġroutes', 'gebra', '386', '_____', '1966', 'Ġfav', 'ussion', 'haw', 'Ġdelusions', 'Dro', 'ĠMcM', 'Ġrock', 'Ġatrocities', '709', 'ĠChild', 'Ġsrc', 'Pol', 'Ġremove', 'Ġphilosopher', '×ľ', 'ĠElse', 'FFER', 'ĠDelaware', 'ĠCoff', 'Ġhomelessness', 'Relations', 'lect', 'Ġstrictly', 'Ġcommands', 'Ġfirms', 'cream', 'asionally', 'Ġclo', 'ĠLys', 'itz', 'ĠSomeone', 'ĠDVDs', 'Ġexciting', 'ista', 'Ġanybody', 'Ġenactment', 'Ġautop', 'Ġfrozen', 'Ġstripping', 've', 'hide', 'Ġsy', 'orne', 'ĠContext', 'Ġbottled', 'Ġracing', 'lishes', 'Ġbreeding', 'ĠBrazilian', 'Ġdemocracy', 'Ġsqu', 'ĠTwenty', 'crafted', 'roo', 'Madison', 'Ġhumans', 'Ġtransition', 'Ġtunnel', 

In [31]:
len(tokenizer.vocab)

50261

In [32]:
len(tokenizer.get_vocab())

50261

In [27]:
tokenizer.get_vocab()

{'ater': 2370,
 'ĠUnknown': 10747,
 'ĠWorking': 33662,
 'attachments': 11190,
 'ck': 366,
 'åıĤèĢĥ': 36880,
 'Ġextmethods': 2735,
 'srcdir': 23944,
 'Ġbenefit': 45761,
 '019': 17813,
 'registrations': 25658,
 'Ġgtf': 26136,
 'Ġkdf': 40291,
 'Ġars': 49328,
 'partial': 4892,
 '3000': 12327,
 'openflow': 15380,
 'shelf': 36158,
 'Tries': 36380,
 'Ġcomparing': 18716,
 'leds': 34414,
 'quential': 12071,
 'Emp': 36682,
 'dci': 38993,
 'Cols': 14608,
 'urable': 11458,
 'ookeeper': 19850,
 'wildcards': 25720,
 'Ġrsync': 30324,
 'ToJson': 39789,
 'Ġmad': 41573,
 'thellier': 44893,
 'hex': 2919,
 'Ġresolvers': 37867,
 'ĠNd': 45605,
 'Ġfolderpath': 48703,
 'jinja': 10721,
 'Preferences': 32740,
 'Ġweb': 4083,
 'diamond': 42666,
 'assignees': 44732,
 'lsh': 48581,
 'ĠgetConstant': 49275,
 ',,,,,,,,': 34214,
 'åħ³': 30468,
 'allocate': 14645,
 'Ġzabbix': 25128,
 'UC': 17387,
 'nothing': 22037,
 '(...': 23952,
 'Ġcourseid': 37319,
 'ier': 2993,
 'mind': 37585,
 'True': 654,
 'cooling': 11321,
 'Ġosc

In [28]:
tokenizer

GPT2TokenizerFast(name_or_path='microsoft/CodeGPT-small-py', vocab_size=50000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|UNKNOWN|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': '<EOL>', 'pad_token': '<pad>'})

In [30]:
tokenizer.vocab['!']

3

## Logit Uploading and Flattening

In [35]:
first_token_distribution.topk(k=1, largest =True) #Returns the most probable token for that position

torch.return_types.topk(
values=tensor([0.2316]),
indices=tensor([65]))

In [47]:
def topk_tuple( logit_vocab_tensor, largest, tokenizer_fn):
    "Run topk for a token"
    #token_position_dict = list( tokenizer.get_vocab().keys() )
    topk = logit_vocab_tensor.topk( k=1 , largest=largest ) #TODO K number of elements can be extended
    #print(topk.indices)
    return ( tokenizer_fn.decode(topk.indices), topk.values.item() )

def min_max_logits( logit_vocab_sample_tensor, tokenizer_fn ):
    "Compute min_max for a sample"
    max_cases = []
    min_cases = []
    for logit_vocab_tensor in logit_vocab_sample_tensor:
        max_cases.append( topk_tuple( logit_vocab_tensor = logit_vocab_tensor, largest = True, tokenizer_fn = tokenizer_fn) ) #TST Max Logit
        min_cases.append( topk_tuple( logit_vocab_tensor = logit_vocab_tensor, largest = False, tokenizer_fn = tokenizer_fn) ) #TST Min Logit
    return max_cases, min_cases

def actual_logit( 
                 logit_vocab_sample_tensor, 
                 tokenized_prompt, 
                 tokenizer_fn,
                 ):
    "Compute actual logits for a sample"
    actual_logits_prompt = []
    for token_pos, id_token in enumerate( tokenized_prompt[1:] ): #Eliminate the first token prediction since we do not use it
        actual_logits_prompt.append(
            (   tokenizer_fn.decode( int(id_token) ), #retrieving the name of the token with the id
                logit_vocab_sample_tensor[token_pos][int(id_token)].item()      ) #retrieving the logit given the position in the sequence and the position in the vocab
            )
    return actual_logits_prompt

In [48]:
max_case,min_case = min_max_logits(
    logit_vocab_sample_tensor = [ soft( torch.from_numpy(token) ) for token in out[:4]],
    tokenizer_fn= tokenizer
    ) #Sentence Level minmax (reduced to 3 tokens)
print(max_case)
print(min_case)

[('_', 0.23164355754852295), ('_', 0.18229301273822784), ('kwargs', 0.5115950107574463), ('):', 0.5035958290100098)]
[('VirtualBox', 2.7581133557608162e-11), ('notices', 7.569445558634064e-14), (' unic', 2.9934349761456683e-15), ('argest', 4.127177961663929e-18)]


In [38]:
sample_id

0

In [45]:
out[0]

array([ 4.3576565, -5.3039317,  3.1336808, ..., -8.81644  , -8.001471 ,
       -4.7138715], dtype=float32)

In [46]:
len(out[0])

50001

In [49]:
actual_cases = actual_logit(
    logit_vocab_sample_tensor = [ soft( torch.from_numpy(token) ) for token in out] , #Out is a complete sequence
    tokenized_prompt = tf_input_ids[ sample_id ],
    tokenizer_fn = tokenizer
    )

In [50]:
actual_cases

[(' install', 4.055943645653315e-05),
 ('(**', 0.00010285110329277813),
 ('kwargs', 0.5115950107574463),
 ('):', 0.5035958290100098),
 ('\n', 1.6293397187006065e-10),
 (' ', 3.421632936806418e-05),
 (' ', 1.2958800255091774e-07),
 (' ', 2.025504919345167e-08),
 (' """', 5.770627725665278e-11),
 ('setup', 0.0005014988710172474),
 (' entry', 0.00017277633014600724),
 (' point', 0.2914642095565796),
 ('"""', 0.3507869839668274),
 ('\n', 1.4548896809518652e-10),
 (' ', 1.5653654372727033e-06),
 (' ', 2.724665293385442e-08),
 (' ', 1.2458069242882175e-08),
 (' if', 1.11607903474642e-10),
 (' USE', 0.00011015804193448275),
 ('_', 0.9874811768531799),
 ('SETUP', 0.46864452958106995),
 ('TOOLS', 0.5258227586746216),
 (':', 0.9227033257484436),
 ('\n', 2.3582106753461396e-11),
 (' ', 1.4689705494674854e-05),
 (' ', 1.6603480617050081e-06),
 (' ', 9.99211692942481e-07),
 (' ', 7.384052764791704e-07),
 (' ', 3.351647421823145e-07),
 (' ', 1.5299599454010604e-07),
 (' ', 7.417709468882094e-08),
 (

In [51]:
print(actual_cases[0])
print(actual_cases[-1])

(' install', 4.055943645653315e-05)
(' )', 2.4953048978204606e-06)


# Logits Execution

In [ ]:
data_pd.shape[0]

In [31]:
def batching_logits( size = 10000 ):

    max_logit_token_prompt = []
    min_logit_token_prompt = []
    actual_logit_token_prompt = []
    soft = torch.nn.Softmax( dim = 0 )                          #Flattening normalization
    
    for file in range( size ):
        out = np.load(params['numpy_files_logits_path']+'/'+ f'logits_tensor[{file}]_batch[{file}]_model[{CODEMODEL}]].npy')
        next_tokens_distribution = [ soft( torch.from_numpy(token) ) for token in out]  #Flattening normalization
        
        max_cases,min_cases = min_max_logits(
            logit_vocab_sample_tensor = next_tokens_distribution,
            tokenizer_fn= tokenizer
            )

        actual_cases = actual_logit(
            logit_vocab_sample_tensor = next_tokens_distribution,
            sample_position = file,
            tokenizer_fn = tokenizer
            )
        
        max_logit_token_prompt.append( max_cases )
        min_logit_token_prompt.append( min_cases )
        actual_logit_token_prompt.append( actual_cases )
        
        logging.info(file)
    return max_logit_token_prompt,min_logit_token_prompt,actual_logit_token_prompt

In [32]:
#max_logit_token_prompt,min_logit_token_prompt,actual_logit_token_prompt = batching_logits( size = 2 )
max_logit_token_prompt,min_logit_token_prompt,actual_logit_token_prompt = batching_logits() #<---WARNING TIME Consuming

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [33]:
dataframe_to_save = data_pd.copy()
dataframe_to_save['max_prob_case'] = max_logit_token_prompt
dataframe_to_save['min_prob_case'] = min_logit_token_prompt
dataframe_to_save['actual_prob_case'] = actual_logit_token_prompt
dataframe_to_save.head()

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids,max_prob_case,min_prob_case,actual_prob_case
0,"def _sample_field(self, sample):\n """"""R...","[('def', 'def', 'function_definition'), ('_sam...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 39873, 62, 3245, 7, 944, 11, 6291...",154,"[(ers, 0.03083224408328533), ((, 0.01664721034...","[(anwhile, 1.0382577819374511e-16), (ousy, 4.6...","[(Ġ_, 0.001834880094975233), (sample, 4.047834..."
1,"def create_cloud_service(self, cloud_service_i...","[('def', 'def', 'function_definition'), ('crea...","[(4299, 'def', 'function_definition'), (2251, ...","[4299, 2251, 62, 17721, 62, 15271, 7, 944, 11,...",459,"[(ers, 0.03083271160721779), (_, 0.34847137331...","[(anwhile, 1.0383171404182079e-16), (ousy, 7.3...","[(Ġcreate, 0.0009968713857233524), (_, 0.34847..."
2,"def clean_dict(dict):\n """"""Remove all keys ...","[('def', 'def', 'function_definition'), ('clea...","[(4299, 'def', 'function_definition'), (3424, ...","[4299, 3424, 62, 11600, 7, 11600, 2599, 198, 2...",151,"[(ers, 0.03083224408328533), (_, 0.18502478301...","[(anwhile, 1.0382577819374511e-16), (ousy, 1.8...","[(Ġclean, 3.9070819184416905e-05), (_, 0.18502..."
3,"def get_pid_from_file(self):\n """"""Get t...","[('def', 'def', 'function_definition'), ('get_...","[(4299, 'def', 'function_definition'), (651, '...","[4299, 651, 62, 35317, 62, 6738, 62, 7753, 7, ...",338,"[(ers, 0.030832389369606972), (_, 0.2419152259...","[(anwhile, 1.0382864354738699e-16), (ousy, 3.2...","[(Ġget, 0.002428646432235837), (_, 0.241915225..."
4,"async def get_artist(self, spotify_id: str) ->...","[('async', 'async', 'function_definition'), ('...","[(292, 'async', 'function_definition'), (13361...","[292, 13361, 825, 651, 62, 49016, 7, 944, 11, ...",185,"[(Ġde, 0.03118526190519333), (., 0.08152643591...","[(arcity, 8.036203538556996e-13), (orgetown, 4...","[(ync, 0.0002533757942728698), (Ġdef, 4.507758..."


In [34]:
parameters['outputs']

'../datax/outputs/out_codesearch_tesbed_EleutherAI-gpt-neo-125M_10000.csv'

In [35]:
## Saving CheckPoint
dataframe_to_save.to_csv( parameters['outputs']  )

In [42]:
dataframe_to_save = pd.read_csv( 
                      parameters['outputs'] , 
                      index_col=0
            )

In [43]:
dataframe_to_save.head()

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids,max_prob_case,min_prob_case,actual_prob_case,loss
0,"def _sample_field(self, sample):\n """"""R...","[('def', 'def', 'function_definition'), ('_sam...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 39873, 62, 3245, 7, 944, 11, 6291...",154,"[('ers', 0.03083224408328533), ('(', 0.0166472...","[('anwhile', 1.0382577819374511e-16), ('ousy',...","[('Ġ_', 0.001834880094975233), ('sample', 4.04...",1.089324
1,"def create_cloud_service(self, cloud_service_i...","[('def', 'def', 'function_definition'), ('crea...","[(4299, 'def', 'function_definition'), (2251, ...","[4299, 2251, 62, 17721, 62, 15271, 7, 944, 11,...",459,"[('ers', 0.03083271160721779), ('_', 0.3484713...","[('anwhile', 1.0383171404182079e-16), ('ousy',...","[('Ġcreate', 0.0009968713857233524), ('_', 0.3...",1.086853
2,"def clean_dict(dict):\n """"""Remove all keys ...","[('def', 'def', 'function_definition'), ('clea...","[(4299, 'def', 'function_definition'), (3424, ...","[4299, 3424, 62, 11600, 7, 11600, 2599, 198, 2...",151,"[('ers', 0.03083224408328533), ('_', 0.1850247...","[('anwhile', 1.0382577819374511e-16), ('ousy',...","[('Ġclean', 3.9070819184416905e-05), ('_', 0.1...",1.241124
3,"def get_pid_from_file(self):\n """"""Get t...","[('def', 'def', 'function_definition'), ('get_...","[(4299, 'def', 'function_definition'), (651, '...","[4299, 651, 62, 35317, 62, 6738, 62, 7753, 7, ...",338,"[('ers', 0.030832389369606972), ('_', 0.241915...","[('anwhile', 1.0382864354738699e-16), ('ousy',...","[('Ġget', 0.002428646432235837), ('_', 0.24191...",0.793477
4,"async def get_artist(self, spotify_id: str) ->...","[('async', 'async', 'function_definition'), ('...","[(292, 'async', 'function_definition'), (13361...","[292, 13361, 825, 651, 62, 49016, 7, 944, 11, ...",185,"[('Ġde', 0.03118526190519333), ('.', 0.0815264...","[('arcity', 8.036203538556996e-13), ('orgetown...","[('ync', 0.0002533757942728698), ('Ġdef', 4.50...",1.092131


## Loss Retrieval

In [38]:
def batching_loss( size = 10000 ):
    output_loss = []
    for current_batch in range(size):
        out = np.load(params['numpy_files_logits_path']+'/'+f'_loss_batch[{current_batch}].npy') 
        output_loss.append( out.item() ) #.item() for numpy library
        print(current_batch)
    return output_loss

In [39]:
output_loss = batching_loss()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [40]:
dataframe_to_save['loss'] = output_loss
dataframe_to_save.head(1)

,whole_func_string,ast_concepts,model_tokenizer_concepts,model_input_ids,model_total_input_ids,max_prob_case,min_prob_case,actual_prob_case,loss
0,"def _sample_field(self, sample):\n """"""R...","[('def', 'def', 'function_definition'), ('_sam...","[(4299, 'def', 'function_definition'), (4808, ...","[4299, 4808, 39873, 62, 3245, 7, 944, 11, 6291...",154,"[('ers', 0.03083224408328533), ('(', 0.0166472...","[('anwhile', 1.0382577819374511e-16), ('ousy',...","[('Ġ_', 0.001834880094975233), ('sample', 4.04...",1.089324


In [41]:
## Saving CheckPoint 2
dataframe_to_save.to_csv( parameters['outputs']  )

# Combining all datasets (last step)